# Hybrid

In [9]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
from math import sqrt
# import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from surprise import SVD

In [10]:
# load dataset
df = pd.read_csv('dataset/movie/ratings.csv')
df_movie = pd.read_csv('dataset/movie/movies.csv')
reader = Reader(rating_scale=(1, 5))
df.drop('timestamp',axis=1)

userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
5            1       70     3.0
6            1      101     5.0
7            1      110     4.0
8            1      151     5.0
9            1      157     5.0
10           1      163     5.0
11           1      216     5.0
12           1      223     3.0
13           1      231     5.0
14           1      235     4.0
15           1      260     5.0
16           1      296     3.0
17           1      316     3.0
18           1      333     5.0
19           1      349     4.0
20           1      356     4.0
21           1      362     5.0
22           1      367     4.0
23           1      423     3.0
24           1      441     4.0
25           1      457     5.0
26           1      480     4.0
27           1      500     3.0
28           1      527     5.0
29           1      543     4.0
...        ...      ...     ...
100812     610   156726     4.5
100813     610   157296     4.0
100814     610   158238     5.0
100815     610   158721     3.5
100816     610   158872     3.5
100817     610   158956     3.0
100818     610   159093     3.0
100819     610   160080     3.0
100820     610   160341     2.5
100821     610   160527     4.5
100822     610   160571     3.0
100823     610   160836     3.0
100824     610   161582     4.0
100825     610   161634     4.0
100826     610   162350     3.5
100827     610   163937     3.5
100828     610   163981     3.5
100829     610   164179     5.0
100830     610   166528     4.0
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0
100836    9999        1     4.0
100837    9999    89745     4.0
100838    9999   102716     4.0
100839    9999   134853     4.0
100840    9999      586     4.0
100841    9999   131130     4.0

[100842 rows x 3 columns]

In [11]:
data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)

In [12]:
data_train = data.build_full_trainset()

In [13]:
algoSVD = SVD(n_epochs=10,lr_all=0.005,reg_all=0.4,)

In [16]:
algoSVD.fit(data_train)

In [34]:
user_id = int(input('Masukan User Id: '))

Masukan User Id: 220


In [35]:
predSVD = pd.DataFrame()
for i in range(1,len(df_movie)):
    predSVD.set_value(i,'movieId',i)
    predSVD.set_value(i,'predRating',algoSVD.predict(user_id,i).est)

C:\Users\okky.qomarudin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\okky.qomarudin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [36]:
final_predSVD = pd.merge(left=predSVD,right=df_movie, left_on='movieId', right_on='movieId')
final_predSVD.sort_values('predRating',ascending=False).head(10)

movieId  predRating                                              title  \
277     318.0    4.342844                   Shawshank Redemption, The (1994)   
602     750.0    4.292270  Dr. Strangelove or: How I Learned to Stop Worr...   
659     858.0    4.275591                              Godfather, The (1972)   
906    1204.0    4.272769                          Lawrence of Arabia (1962)   
2226   2959.0    4.264068                                  Fight Club (1999)   
686     904.0    4.253031                                 Rear Window (1954)   
257     296.0    4.239048                                Pulp Fiction (1994)   
46       50.0    4.225438                         Usual Suspects, The (1995)   
914    1213.0    4.219940                                  Goodfellas (1990)   
224     260.0    4.217687          Star Wars: Episode IV - A New Hope (1977)   

                           genres  
277                   Crime Drama  
602                    Comedy War  
659                   Crime Drama  
906           Adventure Drama War  
2226  Action Crime Drama Thriller  
686              Mystery Thriller  
257   Comedy Crime Drama Thriller  
46         Crime Mystery Thriller  
914                   Crime Drama  
224       Action Adventure Sci-Fi

In [56]:
df_tag = pd.read_csv('dataset/movie/tags.csv')

In [57]:
df_movie = pd.read_csv('dataset/movie/movies.csv')

In [58]:
df_tag.head()

userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200

aggregate tag string into one column 

In [59]:
df_tag = df_tag.groupby(['movieId'])['tag'].apply(' '.join).reset_index()

In [60]:
df_tag.head()

movieId                                           tag
0        1                               pixar pixar fun
1        2  fantasy magic board game Robin Williams game
2        3                                     moldy old
3        5                              pregnancy remake
4        7                                        remake

merge dataset to get genre movie

In [61]:
df_movie['genres'] = df_movie['genres'].str.replace('|',' ',regex=True)
df_tag = pd.merge(df_tag,df_movie,how='left',on='movieId')


In [62]:
df_tag.head()

movieId                                           tag  \
0        1                               pixar pixar fun   
1        2  fantasy magic board game Robin Williams game   
2        3                                     moldy old   
3        5                              pregnancy remake   
4        7                                        remake   

                                title  \
0                    Toy Story (1995)   
1                      Jumanji (1995)   
2             Grumpier Old Men (1995)   
3  Father of the Bride Part II (1995)   
4                      Sabrina (1995)   

                                        genres  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                                       Comedy  
4                               Comedy Romance

In [63]:
# df_tag.drop('timestamp',axis=1,inplace=True)

In [64]:
df_tag[df_tag['movieId']==11]

movieId                 tag                           title  \
5       11  politics president  American President, The (1995)   

                 genres  
5  Comedy Drama Romance

In [65]:
df_tag['description'] = df_tag[['tag', 'genres']].apply(lambda x: ' '.join(x), axis=1)

In [66]:
df_tag[['description']].head()

description
0  pixar pixar fun Adventure Animation Children C...
1  fantasy magic board game Robin Williams game A...
2                           moldy old Comedy Romance
3                            pregnancy remake Comedy
4                              remake Comedy Romance

In [67]:
df_tag.head()

movieId                                           tag  \
0        1                               pixar pixar fun   
1        2  fantasy magic board game Robin Williams game   
2        3                                     moldy old   
3        5                              pregnancy remake   
4        7                                        remake   

                                title  \
0                    Toy Story (1995)   
1                      Jumanji (1995)   
2             Grumpier Old Men (1995)   
3  Father of the Bride Part II (1995)   
4                      Sabrina (1995)   

                                        genres  \
0  Adventure Animation Children Comedy Fantasy   
1                   Adventure Children Fantasy   
2                               Comedy Romance   
3                                       Comedy   
4                               Comedy Romance   

                                         description  
0  pixar pixar fun Adventure Animation Children C...  
1  fantasy magic board game Robin Williams game A...  
2                           moldy old Comedy Romance  
3                            pregnancy remake Comedy  
4                              remake Comedy Romance

In [68]:
#ngram_range untuk kombinasi kata contoh:ngram_range(1,2) very expensive watch -> very expensive, expensive watch
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_tag['description'])

In [69]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in df_tag.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
   similar_items = [(cosine_similarities[idx][i], df_tag['movieId'][i]) for i in similar_indices] 
   results[row['movieId']] = similar_items[1:]

In [70]:
from functools import reduce
from operator import concat

List movie Id from selected user id

In [71]:
final_predSVD['movieId'].astype(int,inplace=True)

0          1
1          2
2          3
3          4
4          5
5          6
6          7
7          8
8          9
9         10
10        11
11        12
12        13
13        14
14        15
15        16
16        17
17        18
18        19
19        20
20        21
21        22
22        23
23        24
24        25
25        26
26        27
27        28
28        29
29        30
        ... 
5371    8957
5372    8958
5373    8959
5374    8961
5375    8964
5376    8965
5377    8966
5378    8967
5379    8968
5380    8969
5381    8970
5382    8972
5383    8973
5384    8974
5385    8977
5386    8978
5387    8979
5388    8981
5389    8982
5390    8983
5391    8984
5392    8985
5393    8987
5394    8989
5395    8998
5396    9004
5397    9005
5398    9008
5399    9010
5400    9018
Name: movieId, Length: 5401, dtype: int32

In [110]:
# final_predSVD['movieId'].astype(int,inplace=True)
list_movie = final_predSVD[ (final_predSVD['predRating']>4.2)]['movieId'].tolist()

In [111]:
list_movie

[50.0,
 260.0,
 296.0,
 318.0,
 750.0,
 858.0,
 904.0,
 912.0,
 1089.0,
 1136.0,
 1197.0,
 1204.0,
 1208.0,
 1213.0,
 1221.0,
 1276.0,
 2959.0,
 4973.0]

### Top 10 Recommended Movie for user ID 2 using Content Base Filtering

In [112]:
keys = list_movie
#find recommended movie on the result
recommended_movie = {item:results.get(item) for item in keys}

recommended_movie = {k: v for k, v in recommended_movie.items() if v is not None}
final_recommended_movie = pd.DataFrame(reduce(concat,recommended_movie.values()),columns={'prediction':'prediction','movieId':'movieId'})
final_recommended_movie = pd.merge(final_recommended_movie,df_movie,how='left',on='movieId').sort_values('prediction',ascending=False)
final_recommended_movie.drop_duplicates().sort_values('prediction',ascending=False).head(20)

prediction  movieId                                              title  \
492     1.000000     1213                                  Goodfellas (1990)   
491     1.000000       16                                      Casino (1995)   
490     1.000000     1466                               Donnie Brasco (1997)   
1470    0.900609      280                         Murder in the First (1995)   
1471    0.693486     7062                         Birdman of Alcatraz (1962)   
493     0.637755     1245                           Miller's Crossing (1990)   
494     0.627300     1221                     Godfather: Part II, The (1974)   
1374    0.627300     1213                                  Goodfellas (1990)   
1372    0.627300       16                                      Casino (1995)   
1373    0.627300      858                              Godfather, The (1972)   
1375    0.627300     1466                               Donnie Brasco (1997)   
1376    0.620762      431                               Carlito's Way (1993)   
882     0.587076     2872                                   Excalibur (1981)   
1177    0.537335     1090                                     Platoon (1986)   
1176    0.537335     1263                            Deer Hunter, The (1978)   
1178    0.537335     3724                                 Coming Home (1978)   
495     0.493887      431                               Carlito's Way (1993)   
1377    0.468233     4262                                    Scarface (1983)   
98      0.464988    33493  Star Wars: Episode III - Revenge of the Sith (...   
99      0.452503     1210  Star Wars: Episode VI - Return of the Jedi (1983)   

                              genres  
492                      Crime Drama  
491                      Crime Drama  
490                      Crime Drama  
1470                  Drama Thriller  
1471                           Drama  
493   Crime Drama Film-Noir Thriller  
494                      Crime Drama  
1374                     Crime Drama  
1372                     Crime Drama  
1373                     Crime Drama  
1375                     Crime Drama  
1376                     Crime Drama  
882                Adventure Fantasy  
1177                       Drama War  
1176                       Drama War  
1178                       Drama War  
495                      Crime Drama  
1377              Action Crime Drama  
98           Action Adventure Sci-Fi  
99           Action Adventure Sci-Fi